In [77]:
from ete2 import NCBITaxa
from Bio import Entrez
import pandas as pd
import json

In [4]:
ncbi = NCBITaxa()

NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...
2092747 names loaded.
252899 synonyms loaded.
Loading nodes...
2092747 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /home/sam/.etetoolkit/taxa.sqlite ...
2092000 generating entries...                                                                                                                                                                                                                                                                                                         

Inserting synonyms:       5000 


Uploading to /home/sam/.etetoolkit/taxa.sqlite



Inserting taxid merges:   50000                                                 

Inserting taxids:       150000            

Inserting taxids:       2090000                                                                                                                                                                                                                                                                                                                                                                                                                            

In [5]:
ncbi.update_taxonomy_database()

Done. Parsing...


Loading node names...
2092747 names loaded.
252899 synonyms loaded.
Loading nodes...
2092747 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /home/sam/.etetoolkit/taxa.sqlite ...
2092000 generating entries...                                                                                                                                                                                                                                                                                                                     
Uploading to /home/sam/.etetoolkit/taxa.sqlite


Inserting synonyms:      10000  

Inserting taxid merges:   50000                                                

Inserting taxids:       250000              

Inserting taxids:       2090000                                                                                                                                                                                                                                                                                                                                                                                                                          

In [48]:
organism = "Teuthida"

In [49]:
taxid2name = ncbi.get_name_translator([organism])
taxid2name

{'Teuthida': [551290]}

Available methods
- NCBITaxa.get_rank()
- NCBITaxa.get_lineage()
- NCBITaxa.get_taxid_translator()
- NCBITaxa.get_name_translator()
- NCBITaxa.translate_to_names()

In [ ]:
descendants = ncbi.get_descendant_taxa(organism, intermediate_nodes=True)
print ncbi.translate_to_names(descendants)

In [47]:
len(descendants)

479

In [64]:
ranks = ncbi.get_rank(descendants)
ranks

{6615: u'family',
 6616: u'genus',
 6618: u'species',
 6622: u'species',
 6623: u'family',
 6624: u'genus',
 6625: u'species',
 6626: u'family',
 6627: u'genus',
 6628: u'species',
 6629: u'species',
 6630: u'genus',
 6631: u'species',
 6632: u'genus',
 6636: u'genus',
 6637: u'species',
 34542: u'suborder',
 34543: u'genus',
 34544: u'species',
 34545: u'genus',
 34546: u'species',
 34547: u'genus',
 34548: u'species',
 34549: u'genus',
 34550: u'species',
 34551: u'species',
 34552: u'genus',
 34553: u'species',
 34554: u'family',
 34555: u'genus',
 34556: u'species',
 34557: u'family',
 34558: u'genus',
 34559: u'species',
 34560: u'family',
 34561: u'genus',
 34562: u'species',
 34563: u'genus',
 34564: u'species',
 34565: u'family',
 34566: u'genus',
 34567: u'species',
 34569: u'genus',
 34570: u'species',
 51370: u'species',
 51371: u'species',
 54069: u'species',
 55283: u'species',
 55284: u'genus',
 55285: u'species',
 55716: u'species',
 55717: u'genus',
 55718: u'species',


In [85]:
df = {}
for taxid in descendants:
    df[taxid] = {}
    
    specie = ncbi.translate_to_names([taxid])
    rank_dict = ncbi.get_rank([taxid])
    lineage_id = ncbi.get_lineage(taxid)
    names = ncbi.get_taxid_translator(lineage_id)
    lineage_name = [names[taxid] for taxid in lineage_id]
    
    df[taxid]['name'] = specie[0]
    df[taxid]['rank'] = rank_dict[taxid]
    df[taxid]['lineage_id'] = '//'.join([str(char) for char in lineage_id])
    df[taxid]['lineage_name'] = '//'.join(lineage_name)

In [86]:
print(json.dumps(df, indent = 2))

{
  "1091609": {
    "lineage_id": "1//131567//2759//33154//33208//6072//33213//33317//1206795//6447//6605//6606//215449//215450//551290//34542//6626//346248//1091609", 
    "name": "environmental samples", 
    "rank": "no rank", 
    "lineage_name": "root//cellular organisms//Eukaryota//Opisthokonta//Metazoa//Eumetazoa//Bilateria//Protostomia//Lophotrochozoa//Mollusca//Cephalopoda//Coleoidea//Neocoleoidea//Decapodiformes//Teuthida//Oegopsina//Ommastrephidae//Dosidicus//environmental samples"
  }, 
  "1091610": {
    "lineage_id": "1//131567//2759//33154//33208//6072//33213//33317//1206795//6447//6605//6606//215449//215450//551290//34542//6626//346248//1091609//1091610", 
    "name": "Dosidicus gigas environmental sample", 
    "rank": "species", 
    "lineage_name": "root//cellular organisms//Eukaryota//Opisthokonta//Metazoa//Eumetazoa//Bilateria//Protostomia//Lophotrochozoa//Mollusca//Cephalopoda//Coleoidea//Neocoleoidea//Decapodiformes//Teuthida//Oegopsina//Ommastrephidae//Dosidicu

In [87]:
data = pd.DataFrame.from_dict(data=df, orient="index")
data.head()
data = data[['name', 'rank', 'lineage_name', 'lineage_id']] # reorder columns

,name,rank,lineage_name,lineage_id
6615,Loliginidae,family,root//cellular organisms//Eukaryota//Opisthoko...,1//131567//2759//33154//33208//6072//33213//33...
6616,Loligo,genus,root//cellular organisms//Eukaryota//Opisthoko...,1//131567//2759//33154//33208//6072//33213//33...
6618,Loligo forbesii,species,root//cellular organisms//Eukaryota//Opisthoko...,1//131567//2759//33154//33208//6072//33213//33...
6622,Loligo vulgaris,species,root//cellular organisms//Eukaryota//Opisthoko...,1//131567//2759//33154//33208//6072//33213//33...
6623,Enoploteuthidae,family,root//cellular organisms//Eukaryota//Opisthoko...,1//131567//2759//33154//33208//6072//33213//33...


In [88]:
data.to_csv(path_or_buf = 'taxonomy_teuthide.csv', index_label='taxid')